In [1]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.messages import BaseMessage
from camel.societies.workforce import Workforce
from camel.toolkits import SearchToolkit
from camel.tasks import Task
from camel.toolkits import FunctionTool
from camel.configs import ZhipuAIConfig
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv())
model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_MAX,
    model_config_dict={"temperature": 0},
    api_key=os.getenv('QWEN_MAX_API_KEY')
)

In [2]:
from camel.agents import ChatAgent
from camel.toolkits import SearchToolkit
toolkit = SearchToolkit()
tools = toolkit.get_tools()
tool_agent = ChatAgent(
    model=model,
    tools=tools,  # List of FunctionTools
)
response = tool_agent.step("A query related to the tool you added")

/workspaces/handy-multi-agent/camel/camel/toolkits/function_tool.py:461: UserWarning: Parameter description is missing for 
                            {'enum': ['searchResults', 'sourcedAnswer', 'structured'], 'type': 'string'}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for


In [3]:
response

ChatAgentResponse(msgs=[BaseMessage(role_name='assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content="Could you please specify which tool you are referring to and what kind of query you would like me to perform? For example, if you're interested in searching for information using a particular search engine, or if you want to look up a specific topic on Wikipedia, let me know the details so I can assist you further.", video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': 'chatcmpl-bcc8885c-a50f-91a1-b1cf-4cd778d86355', 'usage': {'completion_tokens': 64, 'prompt_tokens': 2882, 'total_tokens': 2946, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'termination_reasons': ['stop'], 'num_tokens': 15, 'tool_calls': [], 'external_tool_call_requests': None})

创建workforce

In [ ]:
search_tool = FunctionTool(SearchToolkit.search_google)

#1.创建workforce实例
workforce = Workforce(description="旅游攻略制作与评估组",new_worker_agent_kwargs={'model':model}, coordinator_agent_kwargs={'model':model}, task_agent_kwargs={'model':model})
#2.定义智能体并添加至workforce
search_agent = ChatAgent(
    system_message="""你是一个专业的旅游信息搜索助手。你的职责是:
    1. 搜索目的地的主要景点信息
    2. 搜索当地特色美食信息
    3. 搜索交通和住宿相关信息
    请确保信息的准确性和实用性。""",
    model=model,
    tools=[search_tool],
    output_language='中文'
)
planner_agent = ChatAgent(
    system_message="""你是一个专业的旅游攻略规划师。你的职责是:
    1. 根据用户需求制定详细的旅游计划
    2. 提供景点、餐饮、交通等方面的建议
    3. 确保旅游计划的可行性和合理性
    请确保信息的准确性和实用性。""",
    model=model,
    output_language='中文'
)
reviewer_agent = ChatAgent(
    system_message="""你是一个经验丰富的旅行爱好者。你的职责是:
        1. 从游客角度评估行程的合理性
        2. 指出可能的问题和改进建议
        3. 补充实用的旅行小贴士
        4. 评估行程的性价比
        请基于实际旅行经验给出中肯的建议。""",
    model=model,
    output_language='中文'
)
workforce.add_single_agent_worker("负责搜索目的地相关信息", search_agent).add_single_agent_worker("负责制定详细行程规划",planner_agent).add_single_agent_worker("负责评估行程合理性并提出改进建议", reviewer_agent)
#3.创建任务
from camel.tasks import Task
task = Task(content="规划一个3天的上饶鄱阳旅行计划。", id='0')
#4.workforce处理任务
# 使用jupyter来运行代码请打开下面两行注释
import nest_asyncio
nest_asyncio.apply()
task = workforce.process_task(task)

In [ ]:
task.result